# Cephノード構築： 未使用LVを選択してOSD生成
CepnクラスタにOSDを生成<br>
事前に生成したOSD用のDBとDATA用LVを選択してOSDを生成。

# パラメータ定義

## パラメータファイル設定

Cephクラスタの定義ファイルを指定するため、<br>
パラメータ定義格納ディレクトリを表示する。

In [ ]:
!ls -p ~/notebooks/share/ | grep "/$"

In [ ]:
!ls -p ~/notebooks/share/configuration

In [ ]:
!ls -p ~/notebooks/share/configuration/ceph

In [ ]:
import sys, os.path, importlib, json, yaml

config_file_name = "ceph_admin_vm.yml"
config_file_path = os.path.expanduser("~/notebooks/share/configuration/ceph/{}".format(config_file_name))
with open(config_file_path) as f:
    admin_params = yaml.safe_load(f)
admin_params

In [ ]:
target_vm_name = 'xxx-ceph-admin1'
admin_patam = admin_params[target_vm_name]
admin_patam

In [ ]:
ceph_admin_ip = admin_patam['network']['provisioning']['ip']

In [ ]:
import sys, os.path, importlib, json, yaml

config_file_name = "xxx-openstack-parameter.yml"
config_file_path = os.path.expanduser("~/notebooks/share/configuration/{}".format(config_file_name))
with open(config_file_path) as f:
    params = yaml.safe_load(f)
params

In [ ]:
repo_server = params['repository']['addresses']['service_operation']
repo_server

In [ ]:
provisioning_vip = params['openstack']['provisioning_vip']
provisioning_vip

## CephノードとストレージセグメントIPアドレス対応表の読み込み

In [ ]:
import os
import csv
import pandas as pd

instance_ip_csv_path =  os.environ['HOME'] + "/notebooks/share/configuration/ceph/ceph_instance_ip.csv"
instance_ip_list = []
instance_ip_map = {}

with open(instance_ip_csv_path, 'r') as path_csv:
    reader = csv.DictReader(path_csv)
    
    for record in reader:
        instance_ip_list.append(record)
        instance_ip_map[record['instance_name']] = record

pd.DataFrame(instance_ip_list)

## 作業用ディレクトリを作成する

In [ ]:
import os,tempfile
temp_dir = tempfile.mkdtemp()
print (temp_dir)

## AnsibleのInventoryファイルを作業用ディレクトリへ作成する

In [ ]:
import os

with open( os.path.join(temp_dir, "hosts"), 'w') as f:
    f.write('''[openstack_ctl]
{openstack_cmn}
'''.format(openstack_cmn = provisioning_vip))

hosts = temp_dir + "/hosts"
!cat $hosts

In [ ]:
%env ANSIBLE_INVENTORY={hosts}

疎通確認

In [ ]:
!ansible -m ping openstack_ctl

In [ ]:
!ansible -a 'hostname' openstack_ctl

## OpenStack用OpenRCファイル設定

`xxxxxxx`プロジェクト用のOpenRCファイルを事前にダウンロードしておくこと。  

In [ ]:
import os

user_creds = os.path.expanduser('~/.keys/xxxxxxx-openrc.sh')

assert os.path.exists(user_creds), '{} is not exist'.format(user_creds)

In [ ]:
%env USER_CREDS={user_creds}

In [ ]:
!ls -l {user_creds}

## openstackコマンド用ユーティリティ関数

In [ ]:
def build_cmdline(param):
    cmdline = ''
    
    for param_name, param_value in param.items():
        if isinstance(param_value, dict):
            for key, value in param_value.items():
                cmdline += ' --{} {}={}'.format(param_name, key, value)
        elif isinstance(param_value, list):
            for item in param_value:
                cmdline += ' --{} {}'.format(param_name, item)
        elif isinstance(param_value, bool):
            if param_value:
                cmdline += ' --{}'.format(param_name)
        else:
            cmdline += ' --{} {}'.format(param_name, param_value)

    return cmdline

In [ ]:
import json

def run_openstack(cmd):
    print('EXEC: {}'.format(cmd))
    out = !source {user_creds} && {cmd}
    print('OUTPUT: {}'.format('\n'.join(out)))
    return json.loads('\n'.join(out))

テスト

In [ ]:
run_openstack('openstack flavor list -f json' + build_cmdline({'all': True}))

## デプロイされたCephノード情報の取得

ストレージノードを特定するにあたり、ノード名称が　ceph-nodexxx （xxxの部分は、右寄せ0埋めの数字3桁）というルールに従っていることを前提としている。<br>
 例：ceph-node001<br>
 このノード名称ルールは、構築するCephクラスタ毎に変えるため、以下で定義する。

In [ ]:
# 初期化対象のインスタンス名称を設定する
raise Exception('この先手作業が必要です')

In [ ]:
import pandas as pd
import re

# 対象インスタンス名称マッチングルール
name_match_rule = r'ceph-xxx-node-[0-9]+'

## OpenStack Ironicからノード情報の取得

In [ ]:
cmd = "openstack baremetal node list -f json --long"
nodes = run_openstack(cmd)

In [ ]:
import pandas as pd
import re

for node in nodes:
    instance_uuid = node.get("Instance UUID", None)
    if instance_uuid is None:
        continue
        
    node["instance_name"] = node["Instance Info"]["display_name"]

pd.set_option('display.max_rows', None)
pd.DataFrame(nodes, columns=['Instance UUID', 'instance_name', 'UUID', 'Maintenance', 'Provisioning State', 'Power State', 'Name'])


In [ ]:
cmd = "openstack server list -f json --long"
nova_server_map = run_openstack(cmd)

In [ ]:
node_list = []
for server in nova_server_map:
    node_id = server['ID']
    if re.match(name_match_rule, server['Name']) is None:
        continue
    for node in nodes:
        if node['Instance UUID'] == node_id:
            node_info = {}
            node_list.append(node_info)
            node_info['name'] = server['Name']
            node_info['node_name'] = node['Name']
            instance_ip_info = instance_ip_map.get(server['Name'], None)
            if instance_ip_info is not None:
                node_info['instance_ip'] = instance_ip_info.get('ipv4', None)
            else:
                node_info['instance_ip'] = None
            address = server['Networks']
            prov_net = address['provisioning-net']
            for addr in prov_net:
                node_info['prov_ip'] = addr

pd.DataFrame(node_list)

## 初期化対象Cephノード情報の取得

初期化対象のインスタンス名称を設定する

In [ ]:
# 初期化対象のインスタンス名称を設定する
raise Exception('この先手作業が必要です')

In [ ]:
# bootstrap OSD生成先ノード
instance_name = 'ceph-xxx-node-002'

In [ ]:
bootstrap_node_name = 'ceph-xxx-node-001'

In [ ]:
target = None
for elem in node_list:
    if elem['name'] == instance_name:
        target = elem
        break

assert (target is not None), "指定されたターゲットノードが存在しません。"
print (target)

In [ ]:
bootstrap_node = None
for elem in node_list:
    if elem['name'] == bootstrap_node_name:
        bootstrap_node = elem
        break

assert (bootstrap_node is not None), "指定されたターゲットノードが存在しません。"
print (bootstrap_node)

## 一時Inventoryファイルの生成
以下を実行し、作業用ディレクトリ配下に一時Inventoryファイルを生成する。

Inventoryファイルを生成する。

In [ ]:
node_user='xxxxx'
proxy_user='xxxxx'
proxy_key='~/.ssh/id_ras'

In [ ]:
import os

with open( os.path.join(temp_dir, "hosts"), 'w') as f:
     f.write('''[ceph_admin]
{ceph_admin_ip}   ansible_user={proxy_user} ansible_ssh_private_key_file={proxy_key} ansible_python_interpreter=/usr/bin/python3
[ceph_node]
{node_ip}  ansible_user={node_user} ansible_ssh_common_args='-o ControlMaster=auto -o StrictHostKeyChecking=no -o ControlPersist=30m -o ProxyCommand="ssh -W %h:%p -i {proxy_key} -q {proxy_user}@{ceph_admin_ip}"' ansible_python_interpreter=/usr/bin/python3
'''.format(ceph_admin_ip=ceph_admin_ip, node_ip=target["prov_ip"],
           node_user=node_user, proxy_user=proxy_user, proxy_key=proxy_key))
     f.write('''[bootstrap]
{node_ip}  ansible_user={node_user} ansible_ssh_common_args='-o ControlMaster=auto -o StrictHostKeyChecking=no -o ControlPersist=30m -o ProxyCommand="ssh -W %h:%p -i {proxy_key} -q {proxy_user}@{ceph_admin_ip}"' ansible_python_interpreter=/usr/bin/python3
'''.format(ceph_admin_ip=ceph_admin_ip, node_ip=bootstrap_node["prov_ip"],
           node_user=node_user, proxy_user=proxy_user, proxy_key=proxy_key))

hosts = temp_dir + "/hosts"

!cat $hosts

In [ ]:
%env ANSIBLE_INVENTORY={hosts}

# 疎通確認
以下コマンドを実行し、OpeHubからCephAdmin経由で、Cephノードに疎通することを確認する。

In [ ]:
!ansible -i $hosts -b -m shell -a 'hostname' ceph_admin

以下コマンドを実行し、OpeHubからCephAdminに疎通することを確認する。

In [ ]:
!ansible -i $hosts -b -m shell -a 'hostname' bootstrap

In [ ]:
!ansible -i $hosts -b -m shell -a 'hostname' ceph_node

# 事前PV/LVM情報取得

## LVM情報取得関数定

In [ ]:
import re

def make_pv_info_list(ret):
    ret.pop(0)
    pv_info_list = []
    pv_info = None
    for line in ret:
        if re.search('Physical volume', line) is not None:
            pv_info = {}
        elif re.search('PV Name', line) is not None:
            elems = re.sub('PV Name','',line).split()
            pv_info['pv_name'] = elems[0]
        elif re.search('VG Name', line) is not None:
            elems = re.sub('VG Name','',line).split()
            pv_info['vg_name'] = elems[0]
        elif re.search('PV Size', line) is not None:
            elems = re.sub('PV Size','',line).split()
            pv_info['pv_size'] = elems[0]
            pv_info_list.append(pv_info)

    return pv_info_list

def make_lv_info_list(ret, pv_info_map):
    lv_info_list = []
    lv_info = None
    for line in ret:
        if re.search('Logical volume', line) is not None:
            lv_info = {}
        elif re.search('LV Path', line) is not None:
            elems = re.sub('LV Path','',line).split()
            lv_info['lv_path'] = elems[0]
        elif re.search('LV Name', line) is not None:
            elems = re.sub('LV Name','',line).split()
            lv_info['lv_name'] = elems[0]
        elif re.search('VG Name', line) is not None:
            elems = re.sub('VG Name','',line).split()
            lv_info['vg_name'] = elems[0]
            lv_info['pv_name'] = pv_info_map.get(lv_info['vg_name'],None)
        elif re.search('Block device', line) is not None:
            elems = re.sub('Block device','',line).split()
            lv_info['block_dev'] = elems[0]
            dev_elems = lv_info['block_dev'].split(':')
            dev_name = "dm-{}".format(dev_elems[1])
            lv_info['dev_name'] = dev_name
            lv_info_list.append(lv_info)
            
    return lv_info_list

## PV情報取得

In [ ]:
# get PV Info
ret = !ansible -i $hosts -b -m shell -a 'pvdisplay' ceph_node
pv_info_list = make_pv_info_list(ret)

pv_info_map = {}
for pv_info in pv_info_list:
    vg_name = pv_info['vg_name']
    pv_info_map[vg_name] = pv_info['pv_name']    
    
# get PV Info
ret = !ansible -i $hosts -b -m shell -a 'lvdisplay' ceph_node
lv_info_list = make_lv_info_list(ret,pv_info_map)



In [ ]:
pd.DataFrame(pv_info_list)

In [ ]:
pd.DataFrame(lv_info_list)

In [ ]:
db_lv_info_list = []
data_lv_info_list = []
for lv_info in lv_info_list:
    vg_name = lv_info['vg_name']
    if vg_name == 'vg_db':
        db_lv_info_list.append(lv_info)
        lv_name = lv_info['lv_name']
        lv_id = lv_name.replace('lv_db','')
        lv_info['lv_id'] = int(lv_id)
    elif 'vg_data_' in vg_name:
        data_lv_info_list.append(lv_info)
        lv_name = lv_info['lv_name']
        lv_id = lv_name.replace('lv_data_','')
        lv_info['lv_id'] = int(lv_id)


In [ ]:
pd.DataFrame(db_lv_info_list)

In [ ]:
pd.DataFrame(data_lv_info_list)

# 未使用状態のLVを特定

## 対象OSDの詳細情報取得

In [ ]:
ret = !ansible -i $hosts -b -m shell -a "cephadm shell -- ceph osd metadata --format=json 2> /dev/null" bootstrap

In [ ]:
osd_metadata_list = json.loads(ret[2])
print(json.dumps(osd_metadata_list, indent=2))

In [ ]:
osd_strage_info_list = []
for osd_metadata in osd_metadata_list:
    hostname = osd_metadata['container_hostname']
    if instance_name != hostname:
        continue
    osd_strage_info = {}
    osd_strage_info['hostname'] = hostname
    osd_strage_info['osd_id'] = osd_metadata['id']
    osd_strage_info['bluefs_db_partition_path'] = osd_metadata['bluefs_db_partition_path']
    osd_strage_info['bluefs_db_type'] = osd_metadata['bluefs_db_type']
    osd_strage_info['bluestore_bdev_partition_path'] = osd_metadata['bluestore_bdev_partition_path']
    osd_strage_info['bluestore_bdev_type'] = osd_metadata['bluestore_bdev_type']
    osd_strage_info_list.append(osd_strage_info)

In [ ]:
pd.DataFrame(osd_strage_info_list)

In [ ]:
osd_db_path_map = {}
osd_data_path_map = {}

for osd_strage_info in osd_strage_info_list:
    bluefs_db_partition_path = osd_strage_info['bluefs_db_partition_path']
    bluestore_bdev_partition_path = osd_strage_info['bluestore_bdev_partition_path']
    db_dev = bluefs_db_partition_path.replace('/dev/','')
    data_dev = bluestore_bdev_partition_path.replace('/dev/','')
    osd_db_path_map[db_dev] = osd_strage_info
    osd_data_path_map[data_dev] = osd_strage_info

In [ ]:
print(json.dumps(osd_db_path_map, indent=2))
print(json.dumps(osd_data_path_map, indent=2))


In [ ]:
free_db_lv_info_list = []
free_data_lv_info_list = []

for db_lv_info in db_lv_info_list:
    dev_name = db_lv_info['dev_name']
    if dev_name not in osd_db_path_map:
        free_db_lv_info_list.append(db_lv_info)

for data_lv_info in data_lv_info_list:
    dev_name = data_lv_info['dev_name']
    if dev_name not in osd_data_path_map:
        free_data_lv_info_list.append(data_lv_info)

free_db_lv_info_list = sorted(free_db_lv_info_list, key=lambda x: x['lv_id'])
free_data_lv_info_list = sorted(free_data_lv_info_list, key=lambda x: x['lv_id'])


In [ ]:
pd.DataFrame(free_db_lv_info_list)

In [ ]:
pd.DataFrame(free_data_lv_info_list)

In [ ]:
free_lv_info_map = {}
free_lv_info_list = []

for free_db_lv_info in free_db_lv_info_list:
    lv_id = free_db_lv_info['lv_id']
    free_lv_info_map[lv_id] = {}
    free_lv_info_map[lv_id]['lv_id'] = lv_id
    free_lv_info_map[lv_id]['db_lv_info'] = free_db_lv_info
for free_data_lv_info in free_data_lv_info_list:
    lv_id = free_data_lv_info['lv_id']
    free_lv_id_info = free_lv_info_map.get(lv_id, {})
    free_lv_id_info['lv_id'] = lv_id
    free_lv_id_info['data_lv_info'] = free_data_lv_info

for key,val in free_lv_info_map.items():
    if 'db_lv_info' in val.keys() and 'data_lv_info' in val.keys():
        free_lv_info_list.append(val)

free_lv_info_list = sorted(free_lv_info_list, key=lambda x: x['lv_id'])        
print(json.dumps(free_lv_info_list, indent=2))

# OSD生成

In [ ]:
# 生成するOSD数を指定
raise Exception('この先手作業が必要です')

In [ ]:
apply_osd_num = 12

## DB,DATA LV指定

In [ ]:
apply_lv_info_list = []

if len(free_lv_info_list) < apply_osd_num:
    apply_osd_num = len(free_lv_info_list)

for index in range(apply_osd_num):
    apply_lv_info_list.append(free_lv_info_list[index])


In [ ]:
print(json.dumps(apply_lv_info_list, indent=2))

## OSD構築前のスクラビング無効化

In [ ]:
!ansible -i $hosts -b -m shell -a "cephadm shell -- ceph osd set noscrub" bootstrap

In [ ]:
!ansible -i $hosts -b -m shell -a "cephadm shell -- ceph osd set nodeep-scrub" bootstrap

## バックフィルおよび復元機能を制限

In [ ]:
!ansible -i $hosts -b -m shell -a "cephadm shell -- ceph tell osd.* injectargs --osd-max-backfills 1 --osd-recovery-max-active 1 --osd-recovery-op-priority 1" bootstrap

## OSD 追加

In [ ]:
for index in range(apply_osd_num):
    db_dev = apply_lv_info_list[index]['db_lv_info']['lv_path']
    data_dev = apply_lv_info_list[index]['data_lv_info']['lv_path']
    print("instance_name={} db_dev={} data_dev={}".format(instance_name,db_dev,data_dev))
    !ansible -i $hosts -b -m shell -a "cephadm shell -- ceph orch daemon add osd $instance_name:data_devices=$data_dev,db_devices=$db_dev" bootstrap

In [ ]:
!ansible -i $hosts -b -m shell -a 'cephadm shell -- ceph orch ls 2> /dev/null' bootstrap

In [ ]:
!ansible -i $hosts -b -m shell -a 'cephadm shell -- ceph -s 2> /dev/null' bootstrap

## OSD構築後のスクラビング有効化

In [ ]:
!ansible -i $hosts -b -m shell -a "cephadm shell -- ceph osd unset noscrub" bootstrap

In [ ]:
!ansible -i $hosts -b -m shell -a "cephadm shell -- ceph osd unset nodeep-scrub" bootstrap

## バックフィルおよびリカバリー機能を default に戻す

In [ ]:
!ansible -i $hosts -b -m shell -a "cephadm shell -- ceph tell osd.* injectargs --osd-max-backfills 1 --osd-recovery-max-active 3 --osd-recovery-op-priority 3" bootstrap

## 対象ノードのOSDの詳細情報取得

In [ ]:
ret = !ansible -i $hosts -b -m shell -a "cephadm shell -- ceph osd metadata --format=json 2> /dev/null" bootstrap

In [ ]:
osd_metadata_list = json.loads(ret[2])
print(json.dumps(osd_metadata_list, indent=2))

In [ ]:
osd_strage_info_list = []
for osd_metadata in osd_metadata_list:
    hostname = osd_metadata['container_hostname']
    if instance_name != hostname:
        continue
    osd_strage_info = {}
    osd_strage_info['hostname'] = hostname
    osd_strage_info['osd_id'] = osd_metadata['id']
    osd_strage_info['bluefs_db_partition_path'] = osd_metadata['bluefs_db_partition_path']
    osd_strage_info['bluefs_db_type'] = osd_metadata['bluefs_db_type']
    osd_strage_info['bluestore_bdev_partition_path'] = osd_metadata['bluestore_bdev_partition_path']
    osd_strage_info['bluestore_bdev_type'] = osd_metadata['bluestore_bdev_type']
    osd_strage_info_list.append(osd_strage_info)

In [ ]:
pd.DataFrame(osd_strage_info_list)

In [ ]:
!ansible -i $hosts -b -m shell -a 'cephadm shell -- ceph -s 2> /dev/null' bootstrap

# 後始末

一時ディレクトリを削除する。

In [ ]:
!rm -fr $temp_dir